In [1]:
import sys
import os
from tqdm import tqdm
sys.path.append('/Users/luisgonzales/Thesis/code_base/deep_calibration/rbergomi')
print(os.getcwd())

/Users/luisgonzales/Thesis/code_base/deep_calibration/main


In [2]:
from rbergomi import rBergomi
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from utils import g, b, cov, generate_piecewise_forward_variance, generate_rDonsker_Y
np.random.seed(42)

## Define Grid (like in Horvath et al. 2021)

In [3]:
strikes=np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5 ])
maturities=np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0 ])

## Generate Synthetic MC Data for 80.000 different parameter combinations 

In [9]:
num_combinations = 80000
N = 60000  # number of Monte Carlo paths
n = 100    # time steps per year
T = 2.0    # max maturity
S0 = 1.0
method = 'hybrid'

In [10]:
num_combinations = 50

parameter_combinations = {
    "a": np.random.uniform(-0.49, -0.3, num_combinations),
    "rho": np.random.uniform(-0.99, -0.1, num_combinations),
    "xi_pieces": np.array([np.random.uniform(0.01, 0.1, 8) for _ in range(num_combinations)]),
    "eta": np.random.uniform(0.5, 3.0, num_combinations),
}

xi_curves = []
for xi_vector in parameter_combinations["xi_pieces"]:
    xi_curve, _, _ = generate_piecewise_forward_variance(T=T, n=n, xi_pieces=xi_vector)
    xi_curves.append(xi_curve)
xi_curves = np.array(xi_curves)  # Shape: (80000, 201)
print("xi_curves shape:", xi_curves.shape)
parameter_combinations["xi_curve"] = xi_curves


xi_curves shape: (50, 201)


## Run MC Simulation

In [11]:
option_prices = np.zeros((num_combinations, len(maturities), len(strikes)))
for i in tqdm(range(num_combinations), desc="Simulating parameter combinations..."):
    a = parameter_combinations["a"][i]
    rho = parameter_combinations["rho"][i]
    xi = parameter_combinations["xi_curve"][i]
    eta = parameter_combinations["eta"][i]

    # Generate rBergomi paths
    rB = rBergomi(n=n, N=N, T=T, a=a, method=method)
    dW1 = rB.dW1()
    dW2 = rB.dW2()
    Y = rB.Y(dW1)
    dB = rB.dB(dW1, dW2, rho=rho)
    V = rB.V(Y, xi=xi, eta=eta)
    S = rB.S(V, dB, S0=S0)

    # Loop over each maturity and strike, compute option prices
    for m_idx, T_i in enumerate(maturities):
        t_idx = int(T_i * n)
        S_T = S[:, t_idx]
        for k_idx, K in enumerate(strikes):
            payoff = np.maximum(S_T - K, 0)
            option_prices[i, m_idx, k_idx] = np.mean(payoff)

Simulating parameter combinations...: 100%|██████████| 50/50 [02:36<00:00,  3.13s/it]
